In [9]:
from lib.generator import generate_binary_signal
from lib.channel import apply_channel
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

signal_state = {}
# ===== Slidery =====
Rb_slider = widgets.FloatLogSlider(
    value=10_000,
    base=10,
    min=np.log10(10),     # 10 b/s
    max=np.log10(200_000), # 200 kb/s
    step=0.01,
    description="Rb [b/s]:",
    readout_format='.0f'
)

frame_slider = widgets.IntSlider(
    value=1000,
    min=100,
    max=2000,
    step=10,
    description="Počet bitů:"
)

seed_text = widgets.BoundedIntText(
    value=12345,
    min=1,
    max=99_999,
    step=1,
    description='Seed:'
)

# ===== Tlačítko pro náhodný seed =====
def randomize_seed(b):
    seed_text.value = np.random.randint(1, 100_000)  # automaticky spustí interaktivní aktualizaci

random_seed_button = widgets.Button(description="Random Seed")
random_seed_button.on_click(randomize_seed)

def plot_binary_signal(bits, nrz, t, Rb):
    """
    Zobrazí binární NRZ signál.
    """

    plt.figure(figsize=(12, 4))
    plt.step(t, nrz, where="post")
    plt.ylim(-1.2, 1.2)
    plt.xlabel("Čas [s]")
    plt.ylabel("Amplituda")
    plt.title(f"Telegrafní NRZ signál (Rb = {Rb/1e3:.1f} kb/s)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ===== Funkce pro interaktivní volání =====
def interactive_plot(Rb, frame_length, seed):
    # generování signálu
    bits, nrz, t, fs, Rb_used = generate_binary_signal(
        Rb=int(Rb),
        frame_length=int(frame_length),
        seed=int(seed)
    )
    # ⬇⬇⬇ ULOŽENÍ STAVU ⬇⬇⬇
    signal_state["bits"] = bits
    signal_state["nrz"] = nrz
    signal_state["t"] = t
    signal_state["fs"] = fs
    signal_state["Rb"] = Rb_used

    clear_output(wait=True)  # smaže předchozí grafy při změně
    plot_binary_signal(bits, nrz, t, Rb_used)

    # zobrazení parametrů
    print(f"Seed: {seed}")
    print(f"Vzorkovací frekvence fs = {fs / 1e3:.1f} kHz")
    print(f"Počet bitů = {len(bits)}")

In [11]:
ui = widgets.VBox([Rb_slider, frame_slider, seed_text, random_seed_button])
out = widgets.interactive_output(interactive_plot, {
    'Rb': Rb_slider,
    'frame_length': frame_slider,
    'seed': seed_text
})

display(ui, out)

Output()

In [12]:
B_slider = widgets.FloatLogSlider(
    value=10_000,
    base=10,
    min=2,      # 100 Hz
    max=5.3,    # ~200 kHz
    step=0.1,
    description="B",
    continuous_update=True
)

B_label = widgets.Label()

def format_bandwidth(B):
    if B < 1_000:
        return f"{B:.0f} Hz"
    else:
        return f"{B/1e3:.1f} kHz"

def update_B_label(change):
    B_label.value = f"Šířka pásma: {format_bandwidth(change['new'])}"

B_slider.observe(update_B_label, names='value')

# inicializace
B_label.value = f"Šířka pásma: {format_bandwidth(B_slider.value)}"

SNR_slider = widgets.FloatSlider(
    value=10,
    min=-5,
    max=40,
    step=1,
    description="SNR [dB]",
    continuous_update=True
)

awgn_checkbox = widgets.Checkbox(
    value=True,
    description="AWGN zapnuto"
)

show_noise_checkbox = widgets.Checkbox(
    value=False,
    description="Zobrazit šum"
)


def interactive_channel(B, snr_db, awgn_on, show_noise):
    # ochrana – velmi důležité
    if "nrz" not in signal_state:
        print("⚠️ Nejprve vygeneruj signál")
        return

    nrz = signal_state["nrz"]
    t = signal_state["t"]
    fs = signal_state["fs"]

    channel_out = apply_channel(
        signal=nrz,
        fs=fs,
        B=float(B),
        snr_db=float(snr_db),
        awgn_on=awgn_on
    )
    noise = None
    if awgn_on:
        noise = channel_out - nrz

    plt.step(t, nrz, where="post", alpha=0.4, label="NRZ")
    plt.step(t, channel_out, where="post", label="NRZ + kanál")

    if show_noise and awgn_on:
        plt.step(t, noise, where="post", alpha=0.7, label="Šum")



    plt.figure(figsize=(12, 4))
    plt.step(t, nrz, where="post", alpha=0.4, label="Vstupní NRZ")
    plt.step(t, channel_out, where="post", label="Po kanálu")

    plt.xlabel("Čas [s]")
    plt.ylabel("Amplituda")
    plt.title(
        f"Kanál: B = {B/1e3:.1f} kHz, "
        + (f"SNR = {snr_db:.1f} dB" if awgn_on else "AWGN vypnuto")
    )
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



In [13]:
channel_ui = widgets.VBox([
    B_slider,
    B_label,
    SNR_slider,
    awgn_checkbox,
    show_noise_checkbox
])

channel_out = widgets.interactive_output(
    interactive_channel,
    {
        'B': B_slider,
        'snr_db': SNR_slider,
        'awgn_on': awgn_checkbox,
        'show_noise': show_noise_checkbox
    }
)

display(channel_ui, channel_out)


Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 640x480 with 1 Axes>', 'i…